In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow==1.15
!pip install keras==2.2.5
!pip install git+https://www.github.com/keras-team/keras-contrib.git

     |████████████████████████████████| 412.3MB 32kB/s 
     |████████████████████████████████| 51kB 6.0MB/s 
     |████████████████████████████████| 3.8MB 30.9MB/s 
     |████████████████████████████████| 512kB 41.8MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=8d989e46732ec9a6fca01824255c2ff63ffcc27ae4cf5601d4f9bcccab2a18d4
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully 

In [ ]:
from keras.models import Sequential, Model, Input
from keras.layers import LSTM, Dense, TimeDistributed, Activation, Bidirectional, Masking, Embedding, Dropout, Flatten, concatenate, Conv1D, MaxPool1D, Lambda, Softmax
from keras import backend as K
from keras_contrib.layers import CRF
from keras.utils import plot_model
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras_contrib.losses import  crf_loss
from keras_contrib.metrics import crf_accuracy

Using TensorFlow backend.


In [ ]:
from keras.models import load_model
m = load_model("/content/drive/MyDrive/model/model-ver1.h5")






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




In [ ]:
import numpy as np
from numpy import argmax

In [ ]:
char_encode = np.loadtxt('/content/drive/MyDrive/fast_text/char_encode/char_encode_test_0_3499.txt')
print(char_encode.shape)
char_encode = char_encode.reshape(int(char_encode.shape[0]/42), 42, char_encode.shape[1])
print(char_encode.shape)

(147000, 32)
(3500, 42, 32)


In [ ]:
# chuyển từ char encode sang char embedding
LEN_OF_VOCAB = 137
shape = char_encode.shape
char_embedd = np.zeros([shape[0],shape[1],shape[2],LEN_OF_VOCAB])
for i in range(shape[0]):
  for j in range(shape[1]):
    for k in range(shape[2]):
      char_int = char_encode[i,j,k]
      char_int = char_int.astype(np.int64)
      onehot = np.zeros(LEN_OF_VOCAB)
      onehot[char_int] = 1
      char_embedd[i,j,k,:] = onehot

print(char_embedd.shape)

(3500, 42, 32, 137)


In [ ]:
tag = np.loadtxt('/content/drive/MyDrive/fast_text/tag_embedd/tag_test_0_3499.txt')
tag = tag.reshape(int(tag.shape[0]/42), 42, tag.shape[1])
print(tag.shape)

(3500, 42, 14)


In [ ]:
word = np.loadtxt('/content/drive/MyDrive/fast_text/word_embedd/word_test_0_3499.txt')
word = word.reshape(int(word.shape[0]/42), 42, word.shape[1])
print(word.shape)

(3500, 42, 300)


In [ ]:
y_pred = m.predict([char_embedd, word])
y_true = tag
print(y_pred.shape)
print(y_true.shape)

(3500, 42, 14)
(3500, 42, 14)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
y_pred = y_pred.reshape(y_pred.shape[0]*y_pred.shape[1], y_pred.shape[2])
print(y_pred.shape)
y_pred = np.argmax(y_pred,axis=1)
print(y_pred.shape)

(147000, 14)
(147000,)


In [ ]:
y_true = y_true.reshape(y_true.shape[0]*y_true.shape[1], y_true.shape[2])
print(y_true.shape)
y_true = np.argmax(y_true,axis=1)
print(y_true.shape)

(147000, 14)
(147000,)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
from sklearn.metrics import accuracy_score
acc=0
false=0
for i in range(len(y_true)):
  if y_true[i]!=0:
    if y_true[i]==y_pred[i]:
      acc+=1
    else:
      false+=1
print(acc)
print(false)
print('accuracy = {}'.format(acc/(acc+false)) )

31951
90
accuracy = 0.9971910989045286


In [ ]:
dict = []
for element in y_true:
  if element not in dict:
    dict.append(element)
# print(dict)
dict.sort()
print(dict)

cnf_matrix = confusion_matrix(y_true, y_pred, labels=dict)
for i in range(len(cnf_matrix)):
  for j in range(len(cnf_matrix)):
    cnf_matrix[i][j] = round(cnf_matrix[i][j], 3)
print(cnf_matrix)

arr_to_cal_predict = []
for i in range(len(cnf_matrix)):
  tong= 0
  for j in range(len(cnf_matrix)):
    tong+=cnf_matrix[j][i]
  arr_to_cal_predict.append(tong)
print(arr_to_cal_predict)
# lấy độ predict
predict = []
for i in range(len(cnf_matrix)):
  tong=cnf_matrix[i][i]/arr_to_cal_predict[i]
  predict.append(tong)

print("Độ đo P của từng tag: ")
print(predict)
print(len(predict))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
[[114959      0      0      0      0      0      0      0      0      0
       0      0      0      0]
 [     0   2048     25      0      0      0      0      0      0      0
       0      0      0      0]
 [     0     56   6641      3      0      0      0      0      0      0
       0      0      0      2]
 [     0      0      4    925      0      0      0      0      0      0
       0      0      0      0]
 [     0      0      0      0   1791      0      0      0      0      0
       0      0      0      0]
 [     0      0      0      0      0   1791      0      0      0      0
       0      0      0      0]
 [     0      0      0      0      0      0    893      0      0      0
       0      0      0      0]
 [     0      0      0      0      0      0      0    893      0      0
       0      0      0      0]
 [     0      0      0      0      0      0      0      0     52      0
       0      0      0      0]
 [     0      0      0    

In [ ]:
normalized_confusion_matrix = cnf_matrix/cnf_matrix.sum(axis = 1, keepdims = True)
for i in range(len(normalized_confusion_matrix)):
  for j in range(len(normalized_confusion_matrix)):
    normalized_confusion_matrix[i][j] = round(normalized_confusion_matrix[i][j], 3)
print(normalized_confusion_matrix)

# lấy độ recall
recall = []
for i in range(len(normalized_confusion_matrix)):
  recall.append(normalized_confusion_matrix[i][i])

print("Độ đo R của từng tag: ")
print(recall)
print(len(recall))

[[1.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.   ]
 [0.    0.988 0.012 0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.   ]
 [0.    0.008 0.991 0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.   ]
 [0.    0.    0.004 0.996 0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.   ]
 [0.    0.    0.    0.    1.    0.    0.    0.    0.    0.    0.    0.
  0.    0.   ]
 [0.    0.    0.    0.    0.    1.    0.    0.    0.    0.    0.    0.
  0.    0.   ]
 [0.    0.    0.    0.    0.    0.    1.    0.    0.    0.    0.    0.
  0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.    1.    0.    0.    0.    0.
  0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.    1.    0.    0.    0.
  0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.    0.    1.    0.    0.
  0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    1.    0.
  0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.    0.  

In [ ]:
# lấy độ F1
f1 = []
for i in range(len(recall)):
  p = predict[i]
  r = recall[i]
  f1.append(2*p*r/(p+r))

print("Độ đo F1 của từng tag: ")
print(f1)
print(len(f1))

Độ đo F1 của từng tag: 
[1.0, 0.9806375570909035, 0.9933206399229643, 0.9963834729906861, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9998561565017261]
14
